---
bibliography: bio.bib
csl: harvard-cite-them-right.csl
title: Slap Group Project
execute:
  kernel: python3
format:
  html:
    code-copy: true
    code-link: true
    toc: true
    toc-title: On this page
    toc-depth: 3
    toc_float:
      collapsed: false
      smooth_scroll: true
  pdf:
    include-in-header:
      text: |
        \addtokomafont{disposition}{\rmfamily}
    mainfont: Spectral
    sansfont: Roboto Flex
    monofont: InputMonoCondensed
    papersize: a4
    geometry:
      - top=25mm
      - left=40mm
      - right=30mm
      - bottom=25mm
      - heightrounded
    toc: false
    number-sections: false
    colorlinks: true
    highlight-style: github
jupyter:
  jupytext:
    text_representation:
      extension: .qmd
      format_name: quarto
      format_version: '1.0'
      jupytext_version: 1.16.4
  kernelspec:
    display_name: Python (base)
    language: python
    name: base
---

In [ ]:
#| include: false
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import requests
import zipfile

import matplotlib.cm as cm
import matplotlib.pyplot as plt
from matplotlib.colors import TwoSlopeNorm
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import matplotlib as mpl 

from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy import fcluster, linkage
from scipy.stats import entropy

from dateutil.relativedelta import relativedelta
from datetime import datetime
from PIL import Image
from io import BytesIO
from shapely.geometry import Point
from urllib.parse import urlparse
import base64  # Add this import at the top
import IPython.display as display
import random
from pyproj import Proj, Transformer #for the distance to midpoint
from scipy.stats import shapiro
import scipy.stats as stats
import seaborn as sns
import statsmodels.api as sm
import statsmodels.stats.api as sms
from statsmodels.stats.api import het_breuschpagan
import esda
from splot.esda import moran_scatterplot
from splot.esda import plot_local_autocorrelation
from scipy.stats import skew
from libpysal.weights import Queen
from libpysal import weights
from shapely.ops import unary_union
import contextily as ctx

pd.set_option('display.max_columns', None)
pd.options.display.max_colwidth = 200

In [ ]:
#| include: false

def cache_data(src:str) -> str:
    """Downloads and caches a remote file locally.
    src : str - The remote *source* for the file, any valid URL should work.
    dest : str - The *destination* location to save the downloaded file.    
    Returns a string representing the local location of the file.
    """
    dest = os.path.join('data','raw') #destination to save data
    
    url = urlparse(src) # We assume that this is some kind of valid URL 
    fn = (url.path.split('/')[-3])+(url.path.split('/')[-1]) # return the file name with date
    dfn = os.path.join(dest,fn) # Destination filename
    print(f"Writing to: {fn}")

    if not os.path.isfile(dfn) or os.stat(dfn).st_size < 250:
        
        print(f"{dfn} not found or corrupted, downloading!")
        path = os.path.split(dest)
        if len(path) >= 1 and path[0] != '':
            os.makedirs(os.path.join(*path), exist_ok=True)
            # Download and save the file #os.path.join(*path)
            try:
                with open(dfn, "wb") as file:
                    response = requests.get(src)
                    response.raise_for_status()  # Raise an exception for HTTP errors
                    file.write(response.content)
                print("\tDone downloading.")
            except Exception as e:
                print(f"Error: {e}")
                
        f_size = os.stat(dfn).st_size
        print(f"\tSize is {f_size/1024**2:,.0f} MB ({f_size:,} bytes)")
    else:
        print(f"Found {dfn} locally!")
        f_size = os.stat(dfn).st_size
        print(f"\tSize is {f_size/1024**2:,.0f} MB ({f_size:,} bytes)")
        
    return dfn

In [ ]:
#| include: false

#_____________________________________________________________________________________________________________________________
# Set download URL for London data the date should be choosen by reviewing 
# the InsideAirbnb Get Data page and identifying the date of the required data
# We are using the 2 datasets with earliest and latest scrape of those available
#_____________________________________________________________________________________________________________________________
airbnbdates  = ["2023-12-10", "2024-09-06"]
airbnbcols = ['id', 'listing_url', 'picture_url', 'host_id', 'host_listings_count',
              'host_total_listings_count', 'property_type', 'room_type', 'price', 
              'minimum_nights', 'maximum_nights', 'availability_365', 
              'number_of_reviews', 'latitude', 'longitude', 'last_review', 
              'beds', 'bedrooms', 'host_location']
airbnbdfs = []
for date in airbnbdates:
    airbnburl  = f"https://data.insideairbnb.com/united-kingdom/england/london/{date}/data/listings.csv.gz"
    dfn = cache_data(airbnburl)
    airbnbdfs.append(pd.read_csv(dfn, low_memory=False,usecols=airbnbcols))
   
airbnb_dflist = [airbnb_old, airbnb_new] = airbnbdfs #airbnb_old is the earlier scrape, and new is the later scrape. We don't hardcode the dates for reproducibility with other dates and times.

airbnb_gdfs = []
for df in airbnb_dflist:
#_____________________________________________________________________________________________________________________________
#   I convert the df into gdf, reproject into EPSG 27700
#_____________________________________________________________________________________________________________________________  
    df = gpd.GeoDataFrame(
        df,
        geometry=gpd.points_from_xy(df.longitude, df.latitude),
        crs='EPSG:4326')
    
    df = df.set_crs('EPSG:4326', allow_override=True)
    df.to_crs('EPSG:27700', inplace=True)
    airbnb_gdfs.append(df)

In [ ]:
#| include: false

#_____________________________________________________________________________________________________________________________
# InsideAirbnb data cleaning. We first define the functions to be used in cleaning.
#_____________________________________________________________________________________________________________________________
def pick_active_listings(df, date='str', col="str"):
    """
    Converting review column to datetime, and selecting only the recently active listings. 
    Recently active is defined as those which received at least one review for the year precedeng the scrape.
    Arguments:
    date: string, format "%Y-%m-%d"
    dataframe
    col: string, colname with "last_review" or its alternative
    """
    df.loc[:, col] = pd.to_datetime(df[col])
    date = datetime.strptime(date, "%Y-%m-%d")
    df = df[df['last_review'] > (date - relativedelta(years=1))] #| (df['last_review'].isnull())]
    
    print(f"Data frame after filtering last review date is {df.shape[0]:,} x {df.shape[1]} long.")
    return df

In [ ]:
#| include: false

#_____________________________________________________________________________________________________________________________
# Why do we remove duplicates?
# In the regression part of our analysis, we  want to calculate N of homes that are somehow involved in Airbnb listings, and therefore don't fulle realise it's potential on traditional rental/housing market.
# If we took just listing locations, with the amount of duplicates that we found, the N would show ACTIVITY of hosts more than loss of homes for housing market.
# ChatGPT was used here to determine clustering methods, and for debugging.
#_____________________________________________________________________________________________________________________________

def find_duplicates(df_valid):
    """
    The function looks at hosts with multiple listings and checks if they are within 300m radius (150m max location scattering as per Airbnb's anonymisation algorithm, x2).
    It then estimates the number of genuine homes within the cluster.
    
    Returns a gdf with new columns: 
    'cluster_id' - each unique value is a cluster
    'easting', 'northing' - used in calculating the proximity of listings
    'prvt_rms_in_cluster','entr_hms_in_cluster' - summarises N of private rooms and entire homes within the cluster
    'genuine', 'true_nrooms' - boolean column, and int column, with the following assumptions:

    If the cluster has 0 entire homes, and N rooms, first room will be "converted" to an entire home, and the rest will be discarded. 
    The location of the point will be amended to represent centrepoint of all rooms in the cluster.
    Record of the N of rooms will stay in col "true_nrooms"
    If the cluster has rooms and entire homes, the homes will be treated as genuine homes, and rooms treated as duplicates of the genuine home. 
    With the current timeframes, further enquiries into uniqueness of homes vs rooms inside each cluster is not feasible.
    """
    
    df['easting'], df['northing'] = df.geometry.x, df.geometry.y #this is needed for the clustering  
#_____________________________________________________________________________________________________________________________
#   We are looking for duplicates among multi-listing hosts. First, we filter by new_listings_count > 1.
#_____________________________________________________________________________________________________________________________
    finddups = df_valid[df_valid['new_host_listings_count'] > 1]
    print(f"There are {finddups.shape[0]:,} listings from multi-listing hosts.")
#_____________________________________________________________________________________________________________________________
#   I perform cluster analysis a1nd mark listings from the same host that are within 300m from each other 
#   (150m max location scattering as per Airbnb's anonymisation algorithm, x2). They constitute a cluster, and we then assess if they are a duplicate or not.
#   The selected method computes pairwise distances that can calculate the distance matrix between all points in a group. 
#   Before this method I tried KNN, but the listings were paired incorrectly. 
#   cdist solved this, and it is relatively light on small groups (our dataset is large, but it is split into small groups - by host).
#
#   It is possible that listings outside clusters also could be in one entire home. E.g. there could be several hosts registered for one home.
#   We acknowledge this gap, and advise this for further research.
#_____________________________________________________________________________________________________________________________
    # Process each host group
    for host_id, group in finddups.groupby("host_id"):
        if len(group) > 1:  # Only proceed if there is more than 1 listing
            coords = group[['easting', 'northing']].to_numpy()
            
            # Calculate pairwise distances using pdist (returns condensed distance matrix)
            dist_matrix = pdist(coords)
            
            # Perform hierarchical clustering using the condensed distance matrix
            linkage_matrix = linkage(dist_matrix, method='single')
            clusters = fcluster(linkage_matrix, t=300, criterion='distance')
            
            # Create a series to count cluster sizes
            cluster_sizes = pd.Series(clusters).value_counts()
    
            # Assign clusters to rows in the group, but only for clusters with more than one member - these will retain default column value of None
            finddups.loc[group.index, 'cluster_id'] = [
                f"{host_id}-{cluster}" if cluster_sizes[cluster] > 1 else None
                for cluster in clusters
            ]
#_____________________________________________________________________________________________________________________________
#   We create a subset with non-unique listings only to calculate true_nrooms and determine genuineness.
#   At the very end of the function, this subset is merged back into df_valid.
#_____________________________________________________________________________________________________________________________
    finddups1 = finddups[~finddups['cluster_id'].isna()]
    finddups1 = finddups1.loc[finddups.duplicated(subset='cluster_id', keep=False)]   
    print(f"Found {len(set(finddups1.cluster_id)):,} clusters.")   
#_____________________________________________________________________________________________________________________________
#   I calculate N of entire homes and rooms in each cluster.
#   Limitation found during EDA: "bedrooms" column in InsideAirbnb differs between scrapes. 
#   "2023-12-10" scrape has NaN values in this column, while "2024-09-06" scrape has full data.
#   For both datasets, we use "beds" column as a substitute, because airbnb2024.bedrooms.median()/airbnb2024.beds.median() = 1.
#_____________________________________________________________________________________________________________________________
    finddups1 = finddups.loc[finddups.duplicated(subset='cluster_id',keep=False)]
#   Loop through each cluster by cluster_id. 
    for i, group in finddups1.groupby("cluster_id"):
        # Count number of entire homes and private rooms in the group
        n_entire_homes = group[group['room_type'] == 'Entire home/apt'].shape[0]
        n_private_rooms = group[group['room_type'] == 'Private room'].shape[0]   
        # Assign these counts back to the original DataFrame
        finddups1.loc[group.index, 'entr_hms_in_cluster'] = n_entire_homes
        finddups1.loc[group.index, 'prvt_rms_in_cluster'] = n_private_rooms          
#_____________________________________________________________________________________________________________________________
#   I then determine whether listings are genuine unique homes, or rooms from one entire home.
#   If the cluster has 0 entire homes, and N rooms, first room will be "converted" to an entire home, and the rest will be discarded.
#   Record of the N of rooms will stay in col "true_nrooms".
#   If the cluster has rooms and entire homes, the homes will be treated as genuine homes, and rooms treated as duplicates of the genuine home.
#   It is possible that all entire homes in one cluster are absolutely identical, but with the methods employed we can go this far. We acknowledge the gap and suggest it for further research.
#_____________________________________________________________________________________________________________________________
    for cluster_id, group in finddups1.groupby("cluster_id"):
        
        num_entire_homes = (group['room_type'] == 'Entire home/apt').sum()
        if num_entire_homes == 0:
            # If only rooms are in the cluster
            finddups1.loc[group.index, 'genuine'] = False  # Mark all as duplicates
            finddups1.loc[group.index[0], 'genuine'] = True  # First room becomes the genuine home
#_____________________________________________________________________________________________________________________________
#           For remaining valid homes the geometry is replaced with centroid x and y, taken from points in cluster.  
#           Without this the point and all associated rooms can be aggregated by different spatial unit, considering the scale of the location approximation (up to 150m from origin).
#           This is not crucial for our research, because we aggregate by borough, and potential aggregation errors will be minimal. But for other research with smaller geospatial units it will prove more useful.
#_____________________________________________________________________________________________________________________________
            dissolved_geometry = group.geometry.union_all()  # Combine all geometries in the group
            centroid = dissolved_geometry.centroid # Get the centroid of the combined geometry
            finddups1.at[group.index[0], 'geometry'] = Point(centroid.x, centroid.y)
        else:
            # Cluster has both rooms and entire homes
            finddups1.loc[group.index, 'genuine'] = False  # Default all to duplicates
            entire_home_indices = group[group['room_type'] == 'Entire home/apt'].index
            finddups1.loc[entire_home_indices, 'genuine'] = True  # Mark entire homes with true

    print(f"The clustering exercise identified {len(finddups1[finddups1['genuine'] == False])} listings that are potentially duplicates of other listings.")
    df_valid.update(finddups1)
    
    return df_valid

In [ ]:
#| include: false

airbnb_gdf_clean = []
airbnb_gdf_ingenuine = []
airbnb_gdf_all = []

for df in airbnb_gdfs:
    print(f"Initial dataframe is {df.shape[0]:,} x {df.shape[1]} long.")
    df['price'] = df['price'].fillna('0').str.replace('$', '', regex=False).str.replace(',', '').astype(float).astype(int)
    df.drop(df[df["price"]==0].index, axis=0, inplace=True)
    print(f"Data frame after filtering invalid prices is {df.shape[0]:,} x {df.shape[1]} long.")
    
    df = pick_active_listings(df, date, "last_review")

    ints  = ['id', 'host_listings_count', 'host_id', 'bedrooms', 'beds', 
             'host_total_listings_count', 'minimum_nights', 'maximum_nights', 'availability_365', 'number_of_reviews']
    for i in ints:
        #print(f"Converting {i}")
        try:
            df.loc[:, i] = df[i].fillna(0).astype(float).astype(int)
        except ValueError as e:
            #print("  - !!!Converting to unsigned 16-bit integer!!!")
            df.loc[:, i] = df[i].fillna(0).astype(float).astype(pd.UInt16Dtype())
    #_____________________________________________________________________________________________________________________________
    # After these transformations, the N of listings for each host has changed. We dropped rows where listings were likely inactive at the time of the scrape.
    # We create a new column with host_listings_count, and calculate the counts using grouping by host_id.
    #_____________________________________________________________________________________________________________________________
    host_counts = df['host_id'].value_counts()
    df = pd.merge(df,host_counts,right_index=True,left_on='host_id').rename(columns={'count':'new_host_listings_count'})
        
    # null values in N of bedrooms will interfere with our analysis, therefore we drop them
    df = df[df['beds'].notna()]
    print(f"Data frame after cleaning invalid N of beds is {df.shape[0]:,} listings long.")
    
    # creating columns necessary for the analysis and filling with default values
    df['prvt_rms_in_cluster'] = df['beds']* 0.8 #be are using beds for rooms, but for now just copy the column
    # airbnb_gdfs[1]["bedrooms"].mean()/airbnb_gdfs[1]["beds"].mean() = 0.8
    df['entr_hms_in_cluster'] = 0 #default will be 0
    df['cluster_id'] = None #will write ids here
    #df['true_nrooms'] = df['beds'] #estimation of true n of rooms proved inconclusive with the methods used. The parts related to rooms are commented out.
    df['genuine'] = True   
    
    dups = find_duplicates(df) 
    airbnb_gdf_all.append(dups)
    airbnb_gdf_ingenuine.append((dups[dups['genuine'] == False]).copy()) #collecting False for validation
    
    df = (df[df['genuine'] == True]).copy()
    df = df[(df['availability_365']) > (5.84*12)] #Filtering by "commercial" - rented out more than average per month for London. Average is 5.84 days
#_____________________________________________________________________________________________________________________________
#https://www.ons.gov.uk/peoplepopulationandcommunity/housing/datasets/guestnightsnightsandstaysforshorttermletsuk
#https://www.ons.gov.uk/peoplepopulationandcommunity/housing/datasets/hostslistingsandbedspacesofshorttermletsuk2023
#According to this data, in December 2023 (month where there is overlap between 2 datasets), there were 152,050 nights spent in all 111,880 observed STLs in London, 
#5.84 nights per listing. We take this as a benchmark for "fair" use of Airbnb in London. Hosts that are available for longer than 5.84*12 days will be considereded 
#as those taking advantage of the opportunity, which also means there is a possibility that they don't live in the flat - we will chech this later.
# due to limitations, we can't see the actual occupancy per listing, and have to treat the availability as occupancy. 
# assuming that if the host was non-commercial, they would block out more days when they are living in the flat, and the availability will be lower.
#_____________________________________________________________________________________________________________________________

    print(f"Data frame after cleaning duplicate listings is {df.shape[0]:,} listings long (multi-listing hosts only).\n")
    print(f"Appending cleaned data frame to single-listing hosts data frame, and storing everything in list airbnb_gdf_clean.")
    
    airbnb_gdf_clean.append(df)
    print("Done.\n")

In [ ]:
#| include: false

airbnb_gdf_clean[0].head(3) #this prints 1st element of the list, so the older dataset
#_____________________________________________________________________________________________________________________________
# At this point, airbnb_gdf_clean is a list, which contains [0] - clean dataset of older airbnb data, and [1] - clean newer data
#_____________________________________________________________________________________________________________________________

In [ ]:
#| include: false

#_____________________________________________________________________________________________________________________________
#   You might wonder, whether the find_duplicates function works properly.
#   We acknowledge that, given the timeframes, and considering that our research question is not specifically about duplicates,
#   it is not feasible to develop a complex validation mechanism.
#   However, you can see for yourself the results of the clustering and our assumptions, using the function below.
#_____________________________________________________________________________________________________________________________

# This was not developed as a proper validation tool, only as a proof of concept of finding duplicates.
# As this is not the main part of the exercise, ChatGPT was heavily used here to set up thumbnail plotting.

def check_genuine(df, **kwargs):
    """
    Receives df and random_state as args. If unspecified, gives random number as random_state.
    Displays the random ingenuine listing, and other listings from its cluster as clickable image thumbnails.
    """
    if not kwargs.get('random_state'):
        random_state = random.randint(0,1000)
    else:
        random_state = kwargs.get('random_state')
    print(f'Random_state unspecified, selecting:{random_state}.')
    
    # Select a random "ingenuine" listing from the dataset
    listing_check = df[df['genuine'] == False].sample(1, random_state=random_state)
    
    print(f"Let's have a look at the random listing. ID: {listing_check.id.values[0]}")
    print(f"Cluster number {listing_check.cluster_id.values[0]}")
    
    if (listing_check['beds'] > 4).any():
        print("Our estimate is that this might be a hostel, party home or similar hishly commercialised listing.")
    else:
        print("Our estimate is that this cluster might be one entire home.\n")
    
    print("All listings in this cluster are below. Click on the image to open the listings on website.")
    print("You might get inactive links with past datasets.")
    
    # Get all listings in the same cluster
    same_cluster = df[df["cluster_id"] == listing_check.cluster_id.values[0]]
    # print(same_cluster[["id", "listing_url"]])
    
    # Collect successful images and their associated URLs
    successful_images = []
    urls = []
    
    for idx, row in same_cluster.iterrows():
        try:
            response = requests.get(row['picture_url'], timeout=10)
            response.raise_for_status()  # Raise an error for HTTP issues
            img = Image.open(BytesIO(response.content))
            img.thumbnail((190, 190))  # Resize image for thumbnail display
            successful_images.append((img, row['listing_url']))
        except Exception as e:
            print(f"Failed to retrieve image for Listing ID: {row['id']}. Error: {e}")
    
    # If no successful images, return early
    if not successful_images:
        print("No images could be retrieved.")
        return
    
    # Render images and clickable links in Quarto Markdown
    html_content = "<table>"
    for i, (img, url) in enumerate(successful_images):
        if i % 3 == 0:  # Start a new row every 3 images
            html_content += "<tr>"
        
        # Convert the image to a data URL
        img_buffer = BytesIO()
        img.save(img_buffer, format="PNG")
        img_data = img_buffer.getvalue()
        img_base64 = base64.b64encode(img_data).decode('utf-8')  # Encoding the image
        img_src = f"data:image/png;base64,{img_base64}"
        
        # Add the image and link to the HTML
        html_content += f"""
        <td style="text-align:center; padding:5px">
            <a href="{url}" target="_blank"><img src="{img_src}" style="width:128px; height:auto; border:1px solid #ccc" /></a>
        </td>
        """
        if i % 3 == 2:  # End the row every 3 images
            html_content += "</tr>"
    
    html_content += "</table>"
    
    # Display the HTML in Quarto
    display.display(display.HTML(html_content))

In [ ]:
#| include: false

check_genuine(airbnb_gdf_all[1]) #you can also define random state, e.g. check_genuine(airbnb_gdf_all[1],random_state=10)
#_____________________________________________________________________________________________________________________________
# The algorithm shows 67-74% accuracy, validated with randomised sampling of 25 clusters. See validation sheet in "notebooks", "Cluster validation sheet".
#_____________________________________________________________________________________________________________________________

In [ ]:
#| include: false

#_____________________________________________________________________________________________________________________________
# Loading other data - Csv
#_____________________________________________________________________________________________________________________________
raw_csvs_fn = ['Census_LSOA_Dwellings', 'Census_dwellings' 
    ,'Census_Households', 'Census_Tenure', 'Census_Accommodation', 
    'Census_BedroomOcc', 'Census_CarsVans', 'Census_Deprivation', 'Census_NsSeC', 
    'Census_PopDensity', 'Census_Addresses', 'Sales-2024-09', 
    'First-Time-Buyer-Former-Owner-Occupied-2024-09', 'New-and-Old-2024-09', 'onsrents']
csv_skiprows = [6, 6 ,6, 6, 6, 6, 6, 6, 6, 6, 6, 0, 0, 0, None]
csv_nrows = [4994, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 138543, 60219, 138543, 38390]
csv_parse_dates = [None, None, None, None, None, None, None, None, None, None, None, None, ['Date'], ['Date'], ['Time period']]
csv_usecols = [None, 
    ['local authority: county / unitary (as of April 2023)', 'mnemonic', 'Total'] 
    ,['mnemonic', '2021'],  
    ['mnemonic', 'Owned', 'Shared ownership', 'Social rented', 'Private rented or lives rent free'], 
    ['mnemonic', 'Detached', 'Semi-detached', 'Terraced', 'In a purpose-built block of flats or tenement','Part of a converted or shared house, including bedsits', 'Part of another converted building, for example, former school, church or warehouse', 'In a commercial building, for example, in an office building, hotel or over a shop'], 
    ['mnemonic', 'Occupancy rating of bedrooms: +2 or more', 'Occupancy rating of bedrooms: +1', 'Occupancy rating of bedrooms: 0', 'Occupancy rating of bedrooms: -1', 'Occupancy rating of bedrooms: -2 or less'],
    ['mnemonic', 'No cars or vans in household', '1 car or van in household', '2 cars or vans in household', '3 or more cars or vans in household'], 
    ['mnemonic', 'Household is not deprived in any dimension', 'Household is deprived in one dimension', 'Household is deprived in two dimensions', 'Household is deprived in three dimensions', 'Household is deprived in four dimensions'],
    ['mnemonic', 'L1, L2 and L3 Higher managerial, administrative and professional occupations', 'L4, L5 and L6 Lower managerial, administrative and professional occupations', 
    'L7 Intermediate occupations', 'L8 and L9 Small employers and own account workers', 'L10 and L11 Lower supervisory and technical occupations', 'L12 Semi-routine occupations', 
    'L13 Routine occupations', 'L14.1 and L14.2 Never worked and long-term unemployed', 'L15 Full-time students'], 
    ['mnemonic', '2021'], 
    ['mnemonic', 'Address one year ago is the same as the address of enumeration', 'Address one year ago is student term-time or boarding school address in the UK', 'Migrant from within the UK: Address one year ago was in the UK', 'Migrant from outside the UK: Address one year ago was outside the UK'],
    None,
    ['Date', 'Area_Code', 'First_Time_Buyer_Average_Price', 'Former_Owner_Occupier_Average_Price'],
    ['Date', 'Area_Code', 'New_Build_Average_Price', 'Existing_Property_Average_Price'], 
    None]

csv_dict = {
    "raw_csvs_fn": raw_csvs_fn,
    "csv_skiprows": csv_skiprows,
    "csv_nrows": csv_nrows,
    "csv_parse_dates": csv_parse_dates,
    "csv_usecols": csv_usecols,
}
data_df = []
for idx, item in enumerate(csv_dict["raw_csvs_fn"]):
    df = pd.read_csv(f"data/raw/{item}.csv", 
         skiprows = csv_dict["csv_skiprows"][idx], 
         nrows = csv_dict["csv_nrows"][idx], 
         usecols = csv_dict["csv_usecols"][idx] 
         ,parse_dates = csv_dict["csv_parse_dates"][idx]
    )
    data_df.append(df)

In [ ]:
#| include: false

#_____________________________________________________________________________________________________________________________
# Loading other data - Excel
#_____________________________________________________________________________________________________________________________
raw_excels_fn = [
    'vacant-dwellings-borough.xlsx','hpssadataset9medianpricepaidforadministrativegeographies.xls',
    'hpssadataset15lowerquartilepricepaidforadministrativegeographies.xls',
    'Detailed_LA_20232024.xlsx','privaterentalmarketstatistics231220.xls','privaterentalmarketstatistics231220.xls',
    'privaterentalmarketstatistics231220.xls','privaterentalmarketstatistics231220.xls'  
]
excel_skiprows = [1, 6, 6, 7, 6, 6, 6, 6]
excel_nrows = [34, 331, 331, 310, 357, 357, 357, 357]
excel_sheet_name = [ 2, 10, 10, 3, 12, 13, 14, 15]
excel_usecols = [
[0,21], [2,113], [2,113],[0,4,6,16],['Area Code1', 'Mean', 'Median'],
['Area Code1', 'Mean', 'Median'],['Area Code1', 'Mean', 'Median'],['Area Code1', 'Mean', 'Median']
]
excel_dict = {
    "raw_excels_fn": raw_excels_fn,
    "excel_skiprows": excel_skiprows,
    "excel_nrows": excel_nrows,
    "excel_sheet_name": excel_sheet_name,
    "excel_usecols": excel_usecols,
}
for idx, item in enumerate(excel_dict["raw_excels_fn"]):
    df = pd.read_excel(os.path.join('data','raw', f"{item}"), 
         skiprows = excel_dict["excel_skiprows"][idx], 
         nrows = excel_dict["excel_nrows"][idx], 
         usecols = excel_dict["excel_usecols"][idx] 
         ,sheet_name = excel_dict["excel_sheet_name"][idx]
    )
    first_col_name = df.columns[0]
    df.rename(columns={first_col_name: 'mnemonic'}, inplace=True)
    
    data_df.append(df)

In [ ]:
#| include: false

#_____________________________________________________________________________________________________________________________
# Drop NA and non-London rows. Filter the last step, because first item has codes by LSOA, not by borough
#_____________________________________________________________________________________________________________________________
for idx, df in enumerate(data_df):
    for col in df.columns:
        if isinstance(col, str) and pd.Series(col).str.contains('area.*code', case=False).any():
            df.rename(columns={col: 'mnemonic'}, inplace=True)  
    data_df[idx] = df[df['mnemonic'].notna()]

for idx, df in enumerate(data_df[1:], start=1):
    data_df[idx] = df[df['mnemonic'].str.startswith('E09', na=False)]

In [ ]:
#| include: false

(df_censusLSOAdwell, df_censusdwellings, df_censusHHolds, df_censusTenure, df_censusaccomm, 
df_censusbedocc, df_censuscarsvans, df_censusdepriv, df_censusnssec, df_censuspopdens, df_censusaddress, df_hp_sales, df_hp_firsttime, 
df_hp_newbuild, df_rentalprice, df_vacantprops,df_hp_median,df_hp_lower, df_homless_A2P,df_PRS1bed,df_PRS2bed,df_PRS3bed,df_PRS4bed
) = data_df #We unpack the list, and assign new variable names

In [ ]:
#| include: false

#_____________________________________________________________________________________________________________________________
# Calculate entropy
#_____________________________________________________________________________________________________________________________
for_entropy = [df_censusTenure, df_censusaccomm, df_censusbedocc, df_censuscarsvans, df_censusdepriv, df_censusnssec, df_censusaddress]
for_entropy_strings = ['Tenure_entropy', 'Accommm_entropy', 'Bedocc_entropy', 'Cars_entropy', 'Depriv_entropy', 'NsSEC_entropy', 'Address_entropy']

for idx, i in enumerate(for_entropy):
    array = np.array(i.iloc[:, 1:])  # Select all rows and columns starting from the second column
    entropy_values = np.apply_along_axis(entropy, axis=1, arr=array)  # Apply entropy along each row (axis=1)
    i[f"{for_entropy_strings[idx]}"] = entropy_values

In [ ]:
#| include: false

#_____________________________________________________________________________________________________________________________
# Rename columns for clarity
#_____________________________________________________________________________________________________________________________
df_censusdwellings = df_censusdwellings.rename(
    columns={'local authority: county / unitary (as of April 2023)': 'LAName',
             'Total': 'Total_Dwellings'}
)
df_censusHHolds = df_censusHHolds.rename(columns={'2021': 'Total_Hholds'})
df_censuspopdens = df_censuspopdens.rename(columns={'2021': 'PopDensity'})
df_vacantprops = df_vacantprops.rename(columns={2023: '2023_LTVacant'})
df_hp_median = df_hp_median.rename(columns={'Year ending Mar 2023': 'MedianHP_2023'})
df_hp_lower = df_hp_lower.rename(columns={'Year ending Mar 2023': 'LowerQHP_2023'})

df_homless_A2P = df_homless_A2P.rename(columns={146430: 'HHoldsPrevDuty', 57340: 'HHoldEndAST', 37040: 'HHoldSellRelet'})

df_PRS1bed = df_PRS1bed.rename(columns={'Mean': 'MeanOneBedRent2023', 'Median': 'MedianOneBedRent2023'})
df_PRS2bed = df_PRS2bed.rename(columns={'Mean': 'MeanTwoBedRent2023', 'Median': 'MedianTwoBedRent2023'})
df_PRS3bed = df_PRS3bed.rename(columns={'Mean': 'MeanThreeBedRent2023', 'Median': 'MedianThreeBedRent2023'})
df_PRS4bed = df_PRS4bed.rename(columns={'Mean': 'MeanFourPlusBedRent2023', 'Median': 'MedianFourPlusBedRent2023'})
df_rentalprice = df_rentalprice.rename(columns={'Time period': 'Date'})

In [ ]:
#| include: false

#_____________________________________________________________________________________________________________________________
# Filter sales data for the finantial year
#_____________________________________________________________________________________________________________________________
time_fltr = [df_hp_sales, df_hp_firsttime, df_hp_newbuild, df_rentalprice]
for idx, i in enumerate(time_fltr):
    i["Date"] = pd.to_datetime(i["Date"], errors='coerce')
    time_fltr[idx] = i.query("Date >= '2023-04-01' and Date < '2024-03-31'")
df_hp_sales, df_hp_firsttime, df_hp_newbuild, df_rentalprice = time_fltr

In [ ]:
#| include: false

#_____________________________________________________________________________________________________________________________
# Calculate totals
#_____________________________________________________________________________________________________________________________
monthly_avg_rent = df_rentalprice.groupby(['mnemonic'])['Rental price'].mean().reset_index()
df_hp_firsttime_totals = df_hp_firsttime.groupby(['mnemonic']).mean(['First_Time_Buyer_Average_Price', 'Former_Owner_Occupier_Average_Price'])
df_hp_totals = df_hp_sales.groupby(['mnemonic']).sum(['Sales_Volume'])
df_hp_newbuild_totals = df_hp_newbuild.groupby(['mnemonic']).mean(['New_Build_Average_Price', 'Existing_Property_Average_Price'])

# df_homless_A2P.dropna(how='all', axis=1, inplace=True) 
# df_homless_A2P.dropna(how='all', axis=0, inplace=True) 

In [ ]:
#| include: false
#_____________________________________________________________________________________________________________________________
# Transfrom columns
#_____________________________________________________________________________________________________________________________
df_homless_A2P['HHoldsPrevDuty'] = pd.to_numeric(df_homless_A2P['HHoldsPrevDuty'], errors='coerce')
df_homless_A2P['HHoldEndAST'] = pd.to_numeric(df_homless_A2P['HHoldEndAST'], errors='coerce')
df_homless_A2P['HHoldSellRelet'] = pd.to_numeric(df_homless_A2P['HHoldSellRelet'], errors='coerce')
df_PRS3bed['MeanThreeBedRent2023'] = pd.to_numeric(df_PRS3bed['MeanThreeBedRent2023'], errors='coerce')
df_PRS3bed['MedianThreeBedRent2023'] = pd.to_numeric(df_PRS3bed['MedianThreeBedRent2023'], errors='coerce')
df_PRS4bed['MeanFourPlusBedRent2023'] = pd.to_numeric(df_PRS4bed['MeanFourPlusBedRent2023'], errors='coerce')
df_PRS4bed['MedianFourPlusBedRent2023'] = pd.to_numeric(df_PRS4bed['MedianFourPlusBedRent2023'], errors='coerce')

In [ ]:
#| include: false
#_____________________________________________________________________________________________________________________________
# Creating core datasets to be used in analysis: Housing data
#_____________________________________________________________________________________________________________________________
housing_merge = [df_censusdwellings, df_censusHHolds, df_vacantprops, df_hp_median,
                  df_hp_lower, df_hp_totals, df_homless_A2P, df_hp_firsttime_totals,
                  df_hp_newbuild_totals, df_censusTenure, monthly_avg_rent]

df_CoreHousing = housing_merge[0]
for df in housing_merge[1:]:
    df_CoreHousing = pd.merge(df_CoreHousing, df, on='mnemonic', how='outer')

df_CoreHousing['PrevDutyPercent'] = ((df_CoreHousing['HHoldsPrevDuty']) / (df_CoreHousing['Total_Hholds']))*100
df_CoreHousing['EndASTPercent'] = ((df_CoreHousing['HHoldEndAST']) / (df_CoreHousing['Total_Hholds']))*100
df_CoreHousing['SellReletPercent'] = ((df_CoreHousing['HHoldSellRelet']) / (df_CoreHousing['Total_Hholds']))*100
df_CoreHousing['VacantPercent'] = ((df_CoreHousing['2023_LTVacant']) / (df_CoreHousing['Total_Dwellings']))*100
df_CoreHousing['SalesPercent'] = ((df_CoreHousing['Sales_Volume']) / (df_CoreHousing['Total_Dwellings']))*100
#df_CoreHousing

In [ ]:
#df_censusdwellings['Total_Dwellings'].sum()

In [ ]:
#| include: false
#_____________________________________________________________________________________________________________________________
# Creating core datasets to be used in analysis: Demographics data
#_____________________________________________________________________________________________________________________________
demographics_merge = [df_censusdwellings, df_censusHHolds, df_censusTenure, df_censuspopdens,
                  df_censusaddress, df_censuscarsvans, df_censusdepriv, df_censusnssec,
                  df_censusbedocc, df_censusaccomm]

df_CoreDemographics = demographics_merge[0]
for df in demographics_merge[1:]:
    df_CoreDemographics = pd.merge(df_CoreHousing, df, on='mnemonic', how='outer')
#df_CoreDemographics

In [ ]:
#| include: false
#_____________________________________________________________________________________________________________________________
# Perform some spatial data wrangling on the airbnb data then load spatial data (boroughs, lsoa) and join the airbnb data to the spatial data.
#_____________________________________________________________________________________________________________________________
rents_merge = [df_PRS1bed, df_PRS2bed, df_PRS3bed, df_PRS4bed]
df_PrivateRentsRooms = rents_merge[0]
for df in rents_merge[1:]:
    df_PrivateRentsRooms = pd.merge(df_PrivateRentsRooms, df, on='mnemonic', how='outer')
#df_PrivateRentsRooms

In [ ]:
#| include: false
#_____________________________________________________________________________________________________________________________
# Performing spatial joins of Airbnb data by borough
#_____________________________________________________________________________________________________________________________

# Load London Borough and LSOA shapefiles - boroughs downloaded from "https://data.london.gov.uk/download/statistical-gis-boundary-files-london/9ba8c833-6370-4b11-abdc-314aa020d5e0/statistical-gis-boundaries-london.zip"
boro_shp = os.path.join("data", "london-boundaries", "statistical-gis-boundaries-london", "ESRI", "London_Borough_Excluding_MHW.shp")  
lsoa_shp = os.path.join("data", "lsoa-boundaries", "merged_lsoa_boundaries_london.shp")  

# read shapefile into gdf and reproject crs to 27700
boro_gdf = gpd.read_file(boro_shp).to_crs('EPSG:27700')[['NAME', 'GSS_CODE', 'ONS_INNER', 'geometry']]
lsoa_gdf = gpd.read_file(lsoa_shp).to_crs('EPSG:27700')[['lsoa21cd', 'lsoa21nm', 'lad22cd', 'lad22nm', 'geometry']]

# quickly plot the data
#boro_gdf.plot()
#lsoa_gdf.plot()

In [ ]:
#| include: false
airbnb_gdf_clean[1].columns
#len(airbnb_gdf_clean[1])

In [ ]:
#| include: false
def airbnb_count(gdf, boro_gdf):
    """
    Function to count the airbnb locations within each borough and LSOA, split by room type.
    Args:
    gdf: airbnb gdf
    boro_gdf: geographic areas
    Both gdfs should be in the same projection.
    """ 
    #clip by London boundary
    gdf = gdf.clip(boro_gdf)
    
    # counts the airbnb locations within each borough, split by room type
    # set new gdf for each room type
    entire_ab_gdf = gdf[gdf['room_type'] == "Entire home/apt"]
    pr_ab_gdf = gdf[gdf['room_type'] == "Private room"]
    #other_ab_gdf = gdf[~gdf['room_type'].isin(["Entire home/apt", "Private room"])]
#_____________________________________________________________________________________________________________________________
#Due to the relatively low counts of "other" room types we can discount these from our analysis and focus 
#on "Entire Home/apts" and "Private rooms"
#_____________________________________________________________________________________________________________________________   
    # create point counts per borough
    c1 = boro_gdf.join(gpd.sjoin(gdf, boro_gdf).groupby("index_right").size().rename("total_airbnbs"))
    c2 = c1.join(gpd.sjoin(entire_ab_gdf, c1).groupby("index_right").size().rename("total_entireha"))
    boro_count_gdf = c2.join(gpd.sjoin(pr_ab_gdf, c2).groupby("index_right").size().rename("total_privateroom"))
    #boro_count_gdf = c3.join(gpd.sjoin(other_ab_gdf, c3).groupby("index_right").size().rename("total_other"))
    
    # replaces NaN values with zero
    ## could build funtion to iterate through this
    boro_count_gdf['total_entireha'] = boro_count_gdf['total_entireha'].fillna(0).astype('int64')
    boro_count_gdf['total_privateroom'] = boro_count_gdf['total_privateroom'].fillna(0).astype('int64')
    #boro_count_gdf['total_other'] = boro_count_gdf['total_other'].fillna(0).astype('int64')

    airbnb_count_boro = [boro_count_gdf, entire_ab_gdf, pr_ab_gdf
                         #,other_ab_gdf
                        ]
    
    return airbnb_count_boro

In [ ]:
#| include: false
#_____________________________________________________________________________________________________________________________
# Making the counts of airbnbs
#_____________________________________________________________________________________________________________________________
airbnb_2024_boro = airbnb_count(airbnb_gdf_clean[1], boro_gdf) #list of [boro_count_gdf, entire_ab_gdf, pr_ab_gdf]
airbnb_2023_boro = airbnb_count(airbnb_gdf_clean[0], boro_gdf)
airbnb_2024_lsoa = airbnb_count(airbnb_gdf_clean[1], lsoa_gdf)
airbnb_2023_lsoa = airbnb_count(airbnb_gdf_clean[0], lsoa_gdf)

In [ ]:
#| include: false

len(airbnb_2024_boro[1]), len(airbnb_2023_boro[1]), len(airbnb_2024_lsoa[1]), len(airbnb_2023_lsoa[1])

In [ ]:
#| include: false
#_____________________________________________________________________________________________________________________________
# Creating a basic map of airbnb locations by borough
# lsoa is not being plotted as at this scale it will overcomplicate the map
#_____________________________________________________________________________________________________________________________
fig, ax = plt.subplots(1,1, figsize=(8,6))

# Plot all borough and airbnb data to the same axes
boro_gdf.plot(edgecolor='red', facecolor='none', ax=ax)
airbnb_2024_boro[1].plot(edgecolor='blue', facecolor='none', ax=ax,alpha=0.25, markersize=1.0)
airbnb_2024_boro[2].plot(edgecolor='green', facecolor='none', ax=ax, alpha=0.25, markersize=1.0)
#airbnb_2024_boro[3].plot(edgecolor='orange', facecolor='none', ax=ax, alpha=0.25, markersize=1.0)

# Set the x and y limits
ax.set_xlim(501000, 563000)
ax.set_ylim(155000, 202000)

# Save the image (dpi is 'dots per inch')
#os.makedirs('img', exist_ok=True)
#plt.savefig(os.path.join('img','borough_map_with_air_bnb_locations.png'), dpi=150)

### Growing number of listings
We identified 28,207 active London listings in 2024 - 55.22% year-on-year increase. The listings accounted for 0.48% and 0.74% of total dwellings respectively [@GovUK2012, @powerbiReport2024]; entire homes - 0.35% and 0.58% respectively, in contrast to other sources [@EY_Airbnb_2024]. The central areas exhibit the highest proportion of Airbnb listings relative to total dwellings, and this share increased progressively across these and adjacent boroughs from 2023 to 2024.
### Trend towards entire homes
In 2024, the percentage of entire homes on Airbnb increased to 78%, up from 73% in 2023. The share of private rooms decreased from 26% to 21%, indicating a trend towards more entire homes, which have a greater impact on housing. Some sources suggest STL ease and higher revenue may incentivize removing properties from long-term rental markets [@cromarty2024, @evans2024], even if they start STL as a private room. We therefore see the need to expand the methodology to include private rooms into the research scope.
### More multi-listing hosts
The proportion of multi-listing hosts increased from 53% in 2023 to 57% in 2024, with 36% of these hosts having two listings. The average number of listings per multi-listing host rose from 14.8 to 19.5, and nearly 10% now manage more than 10 listings. Notably, there is growing prevalence of one-bedroom and 3+ bedroom listings, suggesting that multi-listing hosts are increasingly occupying larger homes and one-bedroom properties. This trend may limit access to family-sized housing and properties for first buyers. The high average number of listings likely reflects the involvement of asset management companies, pointing to a trend towards greater commercialisation of the platform.
We compare our observations with the Airbnb-endorsed studies [@EY_Airbnb_2024, @scanlon2024]. As seen from our EDA, the commercial use of the platform is increasing, and the impact on housing could be underestimated. We discuss it further in the next section.


In [ ]:
#| include: false
dwelling_count_2023 = 3790210
#https://assets.publishing.service.gov.uk/media/664da7e6993111924d9d395c/LT_100.ods
dwelling_count_2024 = dwelling_count_2023 + 32162
#net additional dwellings in 2023-2024. https://app.powerbi.com/view?r=eyJrIjoiZTE5YWQ3MDYtZmFjMC00N2YwLWIxM2EtYWY2NTk1NjExYjgwIiwidCI6ImJmMzQ2ODEwLTljN2QtNDNkZS1hODcyLTI0YTJlZjM5OTVhOCJ9

print(f"We have identified {len(airbnb_gdf_clean[0])} valid listings in 2023, and "
      f"{len(airbnb_gdf_clean[1])} valid listings in 2024, with "
      f"{((len(airbnb_gdf_clean[1]) - len(airbnb_gdf_clean[0])) / len(airbnb_gdf_clean[0])) * 100:.2f}% year-on-year increase.")

print(f"The listings accounted for {len(airbnb_gdf_clean[0])/(dwelling_count_2023/100):.2f}% and {len(airbnb_gdf_clean[1])/(dwelling_count_2024/100):.2f}% of total dwellings respectively (Number for London).")
print(f"Entire homes took up {len(airbnb_gdf_clean[0][airbnb_gdf_clean[0]['room_type'] == 'Entire home/apt']) / (dwelling_count_2023) * 100:.2f}% in 2023 and "
      f"{len(airbnb_gdf_clean[1][airbnb_gdf_clean[1]['room_type'] == 'Entire home/apt']) / (dwelling_count_2024) * 100:.2f}% in 2024.\n")

print(f"In 2023, there were:")
print(airbnb_gdf_clean[0]['room_type'].value_counts())
print(f"{len(airbnb_gdf_clean[0][airbnb_gdf_clean[0]['new_host_listings_count'] == 1])} hosts with one listing, and {len(airbnb_gdf_clean[0][airbnb_gdf_clean[0]['new_host_listings_count'] > 1])} hosts with multiple listings.")
print(f"{(len(airbnb_gdf_clean[0][airbnb_gdf_clean[0]['new_host_listings_count'] == 1]) / len(airbnb_gdf_clean[0])) * 100:.2f}% of hosts with one listing, and {(len(airbnb_gdf_clean[0][airbnb_gdf_clean[0]['new_host_listings_count'] > 1]) / len(airbnb_gdf_clean[0])) * 100:.2f}% of hosts with multiple listings.")

print(f"\nIn 2024, there were:")
print(airbnb_gdf_clean[1]['room_type'].value_counts())
print(f"{len(airbnb_gdf_clean[1][airbnb_gdf_clean[1]['new_host_listings_count'] == 1])} hosts with one listing, and {len(airbnb_gdf_clean[1][airbnb_gdf_clean[1]['new_host_listings_count'] > 1])} hosts with multiple listings.")
print(f"{(len(airbnb_gdf_clean[1][airbnb_gdf_clean[1]['new_host_listings_count'] == 1]) / len(airbnb_gdf_clean[1])) * 100:.2f}% of hosts with one listing, and {(len(airbnb_gdf_clean[1][airbnb_gdf_clean[1]['new_host_listings_count'] > 1]) / len(airbnb_gdf_clean[1])) * 100:.2f}% of hosts with multiple listings.")
print("\nDue to the relatively low counts of “other” room types we can discount these from our analysis and focus on “Entire Home/apts” and “Private rooms”.")

In [ ]:
#| include: false
airbnb_gdf_clean[0]['prvt_rms_in_cluster'] = pd.to_numeric(airbnb_gdf_clean[0]['prvt_rms_in_cluster'], errors='coerce')
airbnb_gdf_clean[1]['prvt_rms_in_cluster'] = pd.to_numeric(airbnb_gdf_clean[1]['prvt_rms_in_cluster'], errors='coerce')
plt.figure(figsize=(10, 6))

# Plot the histogram for airbnb_gdf_clean[0] (2023 Listings)
plt.hist(
    airbnb_gdf_clean[0][airbnb_gdf_clean[0]['new_host_listings_count'] > 1]["prvt_rms_in_cluster"].dropna(), 
    bins=range(1, int(airbnb_gdf_clean[0]['prvt_rms_in_cluster'].max()) + 2),  # Ensure bins are integer
    alpha=0.5, label='2023 Listings', color='blue', density=True
)

# Plot the histogram for airbnb_gdf_clean[1] (2024 Listings)
plt.hist(
    airbnb_gdf_clean[1][airbnb_gdf_clean[1]['new_host_listings_count'] > 1]["prvt_rms_in_cluster"].dropna(),
    bins=range(1, int(airbnb_gdf_clean[1]['prvt_rms_in_cluster'].max()) + 2),  # Ensure bins are integer
    alpha=0.5, label='2024 Listings', color='red', density=True
)
plt.xlabel('Number of Bedrooms')
plt.ylabel('Share of Listings, %')
plt.title('N of Bedrooms in Listings from Multi-Listing Hosts')

plt.xticks(range(1, max(int(airbnb_gdf_clean[0]['prvt_rms_in_cluster'].max()), int(airbnb_gdf_clean[1]['beds'].max())) + 1))
plt.legend()
plt.show()

print("We can see that the listings from multi-listing hosts have more rooms in 2021.")

In [ ]:
#| echo: false
multi_listing_hosts_2023 = airbnb_gdf_clean[0][airbnb_gdf_clean[0]['new_host_listings_count'] > 1]
multi_listing_hosts_2024 = airbnb_gdf_clean[1][airbnb_gdf_clean[1]['new_host_listings_count'] > 1]

upper_limit = 10
bins_ml_hosts = list(range(2, upper_limit + 1)) + [upper_limit + 1]

plt.figure(figsize=(10, 6))

plt.hist(
    multi_listing_hosts_2023['new_host_listings_count'], bins=bins_ml_hosts,
    alpha=0.5, label='2023 Listings', color='blue', density=True)

plt.hist(
    multi_listing_hosts_2024['new_host_listings_count'], bins=bins_ml_hosts,
    alpha=0.5, label='2024 Listings', color='red', density=True)

plt.xlabel('Number of Listings per Multi-Listing Host')
plt.ylabel('Share of Multi-Listing Hosts')
plt.title('N of Listings posted by Multi-Listing Hosts')

plt.xticks(list(range(1, upper_limit + 1)) + [upper_limit + 1])
plt.legend()
plt.show()
#___________________________________________________________________________________________________________________________________
# the last bin shows all numbers over 10
#___________________________________________________________________________________________________________________________________


# Print the results
print(f"Average number of listings per multi-listing host in 2023: {multi_listing_hosts_2023['new_host_listings_count'].mean():.2f}")
print(f"Average number of listings per multi-listing host in 2024: {multi_listing_hosts_2024['new_host_listings_count'].mean():.2f}")
print("23% of multi-listing hosts have only 2 listings in 2024. However, the rest have more, and the N of listings per multi-listing host increased from 14.8 to 19.5.")

In [ ]:
#| include: false
# ChatGPT was used here to add additional verification of matching geometry, This is because we are allowing the user to try other InsideAirbnb datasets which we didn't test
def rel_prcnt_change_gdfs(gdf_old, gdf_new, cols):
    """
    Function to find different in number of listings between 2 dates.
    Args:
    gdf_old: earlier airbnb dataset
    gdf_new: latest airbnb dataset
    cols: specify a subset of columns to be substracted
    """
    #Verify that the geometry matches
    if not gdf_old.geometry.equals(gdf_new.geometry):
        raise ValueError("The geometries of the two GeoDataFrames do not match!")
    
    #Verify that the specified columns exist in both GeoDataFrames
    for col in cols:
        if col not in gdf_old.columns or col not in gdf_new.columns:
            raise ValueError(f"Column '{col}' does not exist in both GeoDataFrames.")
    
    #Create a copy of the new gdf to store the differences
    gdf_diff = gdf_new.copy()
    
    # Subtract the specified columns
    for col in cols:
        gdf_diff[col] = ((gdf_new[col] - gdf_old[col]) / gdf_old[col])*100
    
    # Ensure the geometry column is preserved
    gdf_diff['geometry'] = gdf_new['geometry']
    
    return gdf_diff

In [ ]:
#| include: false
cols_diff_airbnb = ['total_airbnbs', 'total_entireha', 'total_privateroom']
diff2024_2023_boro = rel_prcnt_change_gdfs(airbnb_2023_boro[0], airbnb_2024_boro[0], cols_diff_airbnb)
diff2024_2023_lsoa = rel_prcnt_change_gdfs(airbnb_2023_lsoa[0], airbnb_2024_lsoa[0], cols_diff_airbnb)

In [ ]:
#| include: false
diff2024_2023_boro.head(2)

In [ ]:
#| include: false
diff2024_2023_boro["total_airbnbs"].hist()

In [ ]:
#| echo: false
# ChatGPT was used here to plot in grid, and achieve the desired stye
diff2024_2023_boro1 = pd.merge(
    diff2024_2023_boro,  # Left DataFrame
    df_CoreDemographics[['mnemonic', 'Total_Dwellings']],  # Right DataFrame with relevant columns
    left_on='GSS_CODE',  # Column in diff2024_2023_boro
    right_on='mnemonic',  # Column in df_CoreDemographics
    how='left'  # Perform left join to keep all rows from diff2024_2023_boro
)
diff2024_2023_boro1["percent_of_totaldw_2023"] = airbnb_2023_boro[0]['total_airbnbs']/ (diff2024_2023_boro1['Total_Dwellings']/100)
diff2024_2023_boro1["percent_of_totaldw_2024"] = airbnb_2024_boro[0]['total_airbnbs']/ (diff2024_2023_boro1['Total_Dwellings']/100)

# Set the plot's limits for the color scale
vmin = 0
vmax = 2

# Create the figure with two subplots (side-by-side)
fig, axes = plt.subplots(1, 2, figsize=(18, 8))

# Plot the first map for percent_of_totaldw_2023
map_plot_2023 = diff2024_2023_boro1.plot(column='percent_of_totaldw_2023', ax=axes[0], legend=False,
                                          cmap='magma_r', edgecolor='white', linewidth=0.8,
                                          vmin=vmin, vmax=vmax)

# Create the colorbar for the first plot (2023)
sm_2023 = plt.cm.ScalarMappable(cmap='magma_r', norm=mpl.colors.Normalize(vmin=vmin, vmax=vmax))
sm_2023._A = []  # Empty array to create the colorbar
cbar_2023 = fig.colorbar(sm_2023, ax=axes[0], orientation="vertical", fraction=0.02, pad=0.02, shrink=0.6)
cbar_2023.set_label("Percent of Total Dwellings (%)")
cbar_2023.outline.set_visible(False)  # Remove border from the colorbar

# Add labels (percent values) to the first map (2023)
for idx, row in diff2024_2023_boro1.iterrows():
    x, y = row.geometry.centroid.coords[0]
    axes[0].annotate(f"{row['percent_of_totaldw_2023']:.2f}%", xy=(x, y), xytext=(3, 3), textcoords="offset points",
                     ha='center', fontsize=8, color='white', fontweight='bold')

# Set title for the first map
axes[0].set_title("Share of Homes Involved in Airbnb\nin the Total Number of Dwellings (2023)", fontsize=14, loc='left')

# Plot the second map for percent_of_totaldw_2024
map_plot_2024 = diff2024_2023_boro1.plot(column='percent_of_totaldw_2024', ax=axes[1], legend=False,
                                          cmap='magma_r', edgecolor='white', linewidth=0.8,
                                          vmin=vmin, vmax=vmax)

# Create the colorbar for the second plot (2024)
sm_2024 = plt.cm.ScalarMappable(cmap='magma_r', norm=mpl.colors.Normalize(vmin=vmin, vmax=vmax))
sm_2024._A = []  # Empty array to create the colorbar
cbar_2024 = fig.colorbar(sm_2024, ax=axes[1], orientation="vertical", fraction=0.02, pad=0.02, shrink=0.6)
cbar_2024.set_label("Percent of Total Dwellings (%)")
cbar_2024.outline.set_visible(False)  # Remove border from the colorbar

# Add labels (percent values) to the second map (2024)
for idx, row in diff2024_2023_boro1.iterrows():
    x, y = row.geometry.centroid.coords[0]
    axes[1].annotate(f"{row['percent_of_totaldw_2024']:.2f}%", xy=(x, y), xytext=(3, 3), textcoords="offset points",
                     ha='center', fontsize=8, color='white', fontweight='bold')

# Set title for the second map
axes[1].set_title("Share of Homes Involved in Airbnb\nin the Total Number of Dwellings (2024)", fontsize=14, loc='left')

# Set axis limits and turn off axis
for ax in axes:
    ax.set_xlim([501000, 563000])
    ax.set_ylim([155000, 202000])
    ax.set_axis_off()

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
#| include: false
# ChatGPT was used here to plot in grid, and achieve the desired stye
# Ensure the columns exist and calculate vmin and vmax for both datasets
vmin_boro = min(diff2024_2023_boro['total_entireha'].min(), 
                diff2024_2023_boro['total_privateroom'].min(), 
                diff2024_2023_boro['total_airbnbs'].min())
vmax_boro = max(diff2024_2023_boro['total_entireha'].max(), 
                diff2024_2023_boro['total_privateroom'].max(), 
                diff2024_2023_boro['total_airbnbs'].max())

vmin_lsoa = min(diff2024_2023_lsoa['total_entireha'].min(), 
                diff2024_2023_lsoa['total_privateroom'].min(), 
                diff2024_2023_lsoa['total_airbnbs'].min())
vmax_lsoa = max(diff2024_2023_lsoa['total_entireha'].max(), 
                diff2024_2023_lsoa['total_privateroom'].max(), 
                diff2024_2023_lsoa['total_airbnbs'].max())

# Create TwoSlopeNorm instances for custom normalization
norm_boro = TwoSlopeNorm(vmin=-100, vcenter=0, vmax=100)
norm_lsoa = TwoSlopeNorm(vmin=-100, vcenter=0, vmax=100)

# Create the subplots (6 maps: 3 in a row)
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# Plot 'total_airbnbs' for Boroughs with custom normalization
map1 = diff2024_2023_boro.plot(column='total_airbnbs', ax=axes[0, 0], legend=False,
                                cmap='coolwarm', edgecolor='white', norm=norm_boro)
axes[0, 0].set_axis_off()
axes[0, 0].set_title("Relative % Difference\nin Total Airbnb Listings\nby Borough (2024-2023)", fontsize=12, loc='left')
cbar1 = fig.colorbar(map1.get_children()[0], ax=axes[0, 0], orientation="vertical", fraction=0.02, pad=0.02, shrink=0.5)
cbar1.set_label("Relative % Difference", fontsize=10)
cbar1.outline.set_visible(False)

# Plot 'total_entireha' for Boroughs with custom normalization
map2 = diff2024_2023_boro.plot(column='total_entireha', ax=axes[0, 1], legend=False, cmap='coolwarm', edgecolor='white', norm=norm_boro)
axes[0, 1].set_axis_off()
axes[0, 1].set_title("Relative % Difference\nin Entire Home Listings\nby Borough (2024-2023)", fontsize=12, loc='left')
cbar2 = fig.colorbar(map2.get_children()[0], ax=axes[0, 1], orientation="vertical", fraction=0.02, pad=0.02, shrink=0.5)
cbar2.set_label("Relative % Difference", fontsize=10)
cbar2.outline.set_visible(False)

# Plot 'total_privateroom' for Boroughs with custom normalization
map3 = diff2024_2023_boro.plot(column='total_privateroom', ax=axes[0, 2], legend=False, cmap='coolwarm', edgecolor='white', norm=norm_boro)
axes[0, 2].set_axis_off()
axes[0, 2].set_title("Relative % Difference\nin Private Room Listings\nby Borough (2024-2023)", fontsize=12, loc='left')
cbar3 = fig.colorbar(map3.get_children()[0], ax=axes[0, 2], orientation="vertical", fraction=0.02, pad=0.02, shrink=0.5)
cbar3.set_label("Relative % Difference", fontsize=10)
cbar3.outline.set_visible(False)

# Plot 'total_airbnbs' for LSOAs with custom normalization
map4 = diff2024_2023_lsoa.plot(column='total_airbnbs', ax=axes[1, 0], legend=False, cmap='coolwarm', edgecolor='white', linewidth=0.2, norm=norm_lsoa)
axes[1, 0].set_axis_off()
axes[1, 0].set_title("Relative % Difference\nin Total Airbnb Listings\nby LSOA (2024-2023)", fontsize=12, loc='left')
cbar4 = fig.colorbar(map4.get_children()[0], ax=axes[1, 0], orientation="vertical", fraction=0.02, pad=0.02, shrink=0.5)
cbar4.set_label("Relative % Difference", fontsize=10)
cbar4.outline.set_visible(False)

# Plot 'total_entireha' for LSOAs with custom normalization
map5 = diff2024_2023_lsoa.plot(column='total_entireha', ax=axes[1, 1], legend=False,cmap='coolwarm', edgecolor='white', linewidth=0.2, norm=norm_lsoa)
axes[1, 1].set_axis_off()
axes[1, 1].set_title("Relative % Difference\nin Entire Home Listings\nby LSOA (2024-2023)", fontsize=12, loc='left')
cbar5 = fig.colorbar(map5.get_children()[0], ax=axes[1, 1], orientation="vertical", fraction=0.02, pad=0.02, shrink=0.5)
cbar5.set_label("Relative % Difference", fontsize=10)
cbar5.outline.set_visible(False)

# Plot 'total_privateroom' for LSOAs with custom normalization
map6 = diff2024_2023_lsoa.plot(column='total_privateroom', ax=axes[1, 2], legend=False,cmap='coolwarm', edgecolor='white', linewidth=0.2, norm=norm_lsoa)
axes[1, 2].set_axis_off()
axes[1, 2].set_title("Relative % Difference\nin Private Room Listings\nby LSOA (2024-2023)", fontsize=12, loc='left')
cbar6 = fig.colorbar(map6.get_children()[0], ax=axes[1, 2], orientation="vertical", fraction=0.02, pad=0.02, shrink=0.5)
cbar6.set_label("Relative % Difference", fontsize=10)
cbar6.outline.set_visible(False)

# Adjust layout for better spacing
plt.tight_layout()
plt.subplots_adjust(hspace=-0.3, wspace=0.2)  # Adjust gaps between rows and columns
plt.show()

In [ ]:
# #| include: false
# ChatGPT was used here to plot in grid, and achieve the desired stye
# This map is high resolution and therefore commented out. Run this cell if you want to explore localised patterns with Openstreetmap underlay

# # Ensure the LSOA data has the necessary 'total_entireha' column and compute the relative % difference
# # Calculate the vmin and vmax values for custom normalization
# vmin_lsoa = -100
# vmax_lsoa = 100

# # Create TwoSlopeNorm instance for custom normalization
# norm_lsoa = TwoSlopeNorm(vmin=-100, vcenter=0, vmax=100)

# # Create the plot for the LSOA map with relative % difference in 'total_entireha'
# fig, ax = plt.subplots(1, 1, figsize=(15, 10), dpi=300)

# # Plot 'total_entireha' for LSOAs with custom normalization
# map_plot = diff2024_2023_lsoa.plot(column='total_entireha', ax=ax, legend=False,
#                                    cmap='coolwarm', edgecolor='white', linewidth=0.2, norm=norm_lsoa, alpha=0.7)

# # Add OpenStreetMap background using contextily
# ctx.add_basemap(ax, crs=diff2024_2023_lsoa.crs.to_string(), source=ctx.providers.OpenStreetMap.Mapnik, zoom=12)

# # Add colorbar for the map
# cbar = fig.colorbar(map_plot.get_children()[0], ax=ax, orientation="vertical", fraction=0.02, pad=0.02, shrink=0.6)
# cbar.set_label("Relative % Difference", fontsize=10)
# cbar.outline.set_visible(False)  # Remove border from the colorbar

# # Set title for the map
# ax.set_title("Relative % Difference\nin Entire Home Listings\nby LSOA (2024-2023)", fontsize=12, loc='left')

# # Set axis limits and remove axes
# ax.set_axis_off()

# # Show the plot
# plt.tight_layout()
# plt.show()

In [ ]:
# #| echo: false
# # write the data to be used in further analysis (ONLY IF NEEDED. CODE WORKS WITHOUT THIS)
# fn = "borough_counts.csv"
# path = os.path.join('data','clean')

# if not os.path.exists(path):
#     print(f"Creating {path} under {os.getcwd()}")
#     os.makedirs(path)
    
# boro_count_gdf.to_csv(os.path.join(path,fn), index=False)
# #boro_count_gdf.to_file("boro_count.geoparquet")
# print("Done.")

# fn = "lsoa_counts.csv"
# path = os.path.join('data','clean')

# if not os.path.exists(path):
#     print(f"Creating {path} under {os.getcwd()}")
#     os.makedirs(path)
    
# lsoa_count_gdf.to_csv(os.path.join(path,fn), index=False)
# #lsoa_count_gdf.to_file("lsoa_count.geoparquet")
# print("Done.")

## 7. Drawing on your previous answers, and supporting your response with evidence (*e.g.* figures, maps, EDA/ESDA, and simple statistical analysis/models drawing on experience from, e.g., CASA0007), how *could* the InsideAirbnb data set be used to inform the regulation of Short-Term Lets (STL) in London? 

### Potential link between STL and housing crisis in London
Despite a narrowing rent-wage gap, rising living costs force low-income Londoners to spend up to 77% of wages on housing. Higher mortgage rates further exclude many from homeownership, disproportionately impacting vulnerable populations, leading to poverty, evictions, overcrowding, or homelessness [@EY_Airbnb_2024, @scanlon2024, @finnerty2024, @otte2024, @builtassetmanagement2024, @londonassembly2024, @ageuk2024, @kellyk2024, @crisis2024, @bevan2023]. There are many claims that STLs exacerbate this housing crisis, pushing the demand down to the social and private rental sector.
In February 2024, the UK government announced reforms requiring planning permission for STLs exceeding 90 nights, and a mandatory STL registration scheme [@govuk2024] that will close the STL data availability gap, discussed in Q4. Additionally, the furnished holiday lettings tax regime will be abolished [@govuk_abolition2024]. These continuous efforts aim to keep homes available for local communities.
Airbnb claims no link between listing growth and rising housing costs, backed by studies discussed in Q6. A caveat of published studies is their UK-wide statistics for some parameters, which may not reflect London’s heated STL market. With Airbnb comprising 47% of European platform STL supply and 30% across all types, STL impacts on housing may be more significant [@lighthouse2021, @laymyhat].
### Our research
We base our research on the independent studies conducted in 2019 and 2020 [@Shabrina2021, @Todd2021]. Our goal is to identify proxies accounting for the impacts of the Airbnb listings on housing in London. We utilise the methodology outlined in the articles above.
### Datasets:
* InsideAirbnb - see Q6 for details.
* London Boroughs' administrative boundaries provide a foundation for decision-makers to conduct further analysis.
* LSOA 2021 represent smaller geographies. Issues include potential misrepresentation of demographics, exclusion of daytime populations, and being too granular for effective citywide visualisation.
* Census 2021 provides the most accurate population data, but results may be skewed due to limited mobility and migration during the Covid-19 pandemic.

### Results and interpretation:

The model explains about 53% of the variation in rental prices (R²), offering a solid understanding of the main factors influencing rent changes.


In [ ]:
#| include: false

#___________________ extracting data needed for regression _______________
# Extracting the percentage and the total number of vacant properties in 2023
For_regressionV = df_CoreDemographics[['VacantPercent','2023_LTVacant', 'LAName', 'mnemonic']]

# Display the first few rows to ensure it worked correctly
print(For_regressionV.head())

#print(df_rentalprice) 

# Extracting Annual change column because we want to observe how the number of listings may influence the change in rental price
For_RegressionR = df_rentalprice[['mnemonic', 'Area name', 'Annual change']]

#print(For_RegressionR)

# ______________________ calculating the distance to the centre _________

# defining latitudes and longitude finder from london uk 
lat_lon = [
    (51.5014, -0.1419),  # Buckingham Palace
    (51.5081, -0.0760),  # Tower of London
    (51.5194, -0.1269),  # British Museum
    (51.5007, -0.1246),  # Big Ben
    (51.5033, -0.1195)   # London Eye
]

# Reprojecting coordinates to BNG (EPSG:27700)
transformer = Transformer.from_crs("epsg:4326", "epsg:27700", always_xy=True)
eastings_northings = [transformer.transform(lon, lat) for lat, lon in lat_lon]

# Calculating midpoint
eastings, northings = zip(*eastings_northings)
midpoint = (np.mean(eastings), np.mean(northings))


# Calculating centroids of each borough and distance to midpoint
boro_gdf["centroid"] = boro_gdf.geometry.centroid
boro_gdf["distance_to_midpoint"] = boro_gdf["centroid"].apply(
    lambda point: np.sqrt((point.x - midpoint[0])**2 + (point.y - midpoint[1])**2)
)


# Display only the NAME and distance_to_midpoint columns
#print(boro_gdf[['GSS_CODE', 'distance_to_midpoint', 'geometry']])

#print(diff2024_2023_boro)

In [ ]:
#| include: false


#shapiro wilk test to see if the data is normally distributed
#-------  on the listings data ---------

# Perform the Shapiro-Wilk test
stat, p_value = shapiro(diff2024_2023_boro["total_airbnbs"])

# Display the results
print(f"Shapiro-Wilk Test Statistic: {stat}")
print(f"P-value: {p_value}")


#the W values 0.7... is less than 1 (suggesting that the data is not normally distributed) and the p value 0.00000505 is <0.05
#therefore the null hypothesis which assumes that the data is normally distributed is rejected 



# ----------- Perform the test on the '2023' column ------ #
stat, p_value = shapiro(For_regressionV["VacantPercent"])
print(f"Shapiro-Wilk Test Statistic: {stat}")
print(f"P-value: {p_value}")

#Interpretation 
#0.9465112270729066 W value close to one so its close to normal 
#p value is 0.10520533110791175 greater than 0.05 so we can fail to reject the null hypothesis as there is no significant evidence
#that the data is not normally distributes



# Create Q-Q plot
stats.probplot(For_regressionV["VacantPercent"], dist="norm", plot=plt)
plt.show()

#hist to actually visualise the distribution to determine whether we transform or not
# Histogram with a density line
sns.histplot(For_regressionV["VacantPercent"], kde=True, color="blue", bins=20)

# Adding labels and title
plt.xlabel('vacant properties')
plt.ylabel('Density')
plt.title('Histogram vacant properties')

# Show plot
plt.show()

#as you can see it is still a bit skewed to the right



#Now we'll do one for the rental price
stat, p_value = shapiro(For_RegressionR['Annual change'])
print(f"Shapiro-Wilk Test Statistic: {stat}")
print(f"P-value: {p_value}")

#owned
stat, p_value = shapiro(boro_gdf['distance_to_midpoint'])
print(f"Shapiro-Wilk Test Statistic: {stat}")
print(f"P-value: {p_value}")


# checking ifits skewed
from scipy.stats import skew

skew_vac = skew(For_regressionV['VacantPercent'])


print(f"Skewness of Rented: {skew_vac}")

In [ ]:
#| include: false


# Histograms for all variables to visualise distribution 
# Histogram for "total_airbnbs"
sns.histplot(diff2024_2023_boro["total_airbnbs"], kde=True, color="blue", bins=20)
plt.xlabel('Total Airbnbs')
plt.ylabel('Density')
plt.title('Histogram of Total Airbnbs')
plt.show()

# Histogram for "VacantPercent"
sns.histplot(For_regressionV["VacantPercent"], kde=True, color="green", bins=20)
plt.xlabel('Vacant Percent')
plt.ylabel('Density')
plt.title('Histogram of Vacant Percent')
plt.show()

# Histogram for "Annual Change"
sns.histplot(For_RegressionR['Annual change'], kde=True, color="orange", bins=20)
plt.xlabel('Annual Change')
plt.ylabel('Density')
plt.title('Histogram of Annual Change')
plt.show()

# Histogram for "distance_to_midpoint"
sns.histplot(boro_gdf['distance_to_midpoint'], kde=True, color="purple", bins=20)
plt.xlabel('Distance to Midpoint')
plt.ylabel('Density')
plt.title('Histogram of Distance to Midpoint')
plt.show()

In [ ]:
#| include: false


#______ ________ merging the data for the regression model __________



# Merge using different column names
vacantprops_airbnb = pd.merge(For_regressionV[['mnemonic', 'LAName', 'VacantPercent']],  
                                   diff2024_2023_boro[['GSS_CODE', 'total_airbnbs']], 
                                   left_on='mnemonic',  
                                   right_on='GSS_CODE',  
                                   how='left', suffixes=('', '_dup'))


# Print the first few rows of the merged DataFrame
#print(vacantprops_airbnb.head())


#Adding rents
Adding_more = pd.merge(
    vacantprops_airbnb, 
    For_RegressionR[['mnemonic', 'Annual change']], 
    left_on='mnemonic',  # Merge on LAcode from vacantprops_with_tenure
    right_on='mnemonic',  # Merge on GSS_CODE from merged_borough_counts
    how='inner'  # Inner join to only keep matching rows
)

# Merging the result with boro to add Distance to Midpoint
Regression_Final = pd.merge(
    Adding_more, 
    boro_gdf[['GSS_CODE', 'distance_to_midpoint', 'geometry']], 
    on='GSS_CODE',  # Merge on GSS_CODE which is common
    how='inner'  # Inner join to keep only matching rows
)

# Display the final merged dataframe
#print(Regression_Final)


# Removing duplicate columns if any
Regression_Final = Regression_Final.loc[:, ~Regression_Final.columns.duplicated()]


# Dropping the redundant 'mnemonic' column after merging
Regression_Final1 = Regression_Final.drop(columns=['mnemonic'], errors='ignore')

# Print the final DataFrame
#print(Regression_Final1.head())

#______ grouping the rows by borough ______


# Group by borough and aggregate
Regression_Final1 = Regression_Final.groupby('LAName').agg({
    'VacantPercent': 'mean',
    'total_airbnbs': 'mean',
    'Annual change': 'mean',
    'distance_to_midpoint': 'mean',
}).reset_index()

# Combine geometries for each group
geometry = Regression_Final.groupby('LAName')['geometry'].apply(lambda g: unary_union(g)).reset_index()

# Merge geometries back into the grouped data
Regression_Final1 = Regression_Final1.merge(geometry, on='LAName')

# Convert to GeoDataFrame
Regression_Final1 = gpd.GeoDataFrame(Regression_Final1, geometry='geometry')

# Convert distance to kilometers
Regression_Final1['distance_to_midpoint_km'] = Regression_Final1['distance_to_midpoint'] / 1000

print(Regression_Final1.head())



Regression_Final1.loc[:, 'distance_to_midpoint_km'] = Regression_Final1['distance_to_midpoint'] / 1000
#the scale should not dominate others numerically which is why the distance to the centre is being calculated in km

#Viewing the new column to ensure the operation worked
Regression_Final1[['distance_to_midpoint', 'distance_to_midpoint_km']].head()

In [ ]:
#| include: false

# __________________ Running the regression model ____________


# Running the first regression model on the data
X = Regression_Final1[['total_airbnbs','VacantPercent','distance_to_midpoint_km']]  
y = Regression_Final1['Annual change']  


# Fitting the model
model = sm.OLS(y, sm.add_constant(X)).fit()

# Displaying model summary
print(model.summary())

# Plot residuals after the transformation
residuals = model.resid
fitted_values = model.fittedvalues

# ______________ Residuals vs Fitted plot
'''
plt.scatter(fitted_values, residuals)
plt.axhline(0, color='black', linestyle='--')
plt.title("Residuals vs Fitted Values")
plt.xlabel("Fitted Values")
plt.ylabel("Residuals")
plt.show()
'''
# QQ plot
#sm.qqplot(residuals, line='45')
#plt.show()


# Breusch-Pagan test for heteroskedasticity
_, p_value, _, _ = sms.het_breuschpagan(residuals, model.model.exog)
#print(f'Breusch-Pagan p-value: {p_value}')

from statsmodels.stats.diagnostic import het_white

# White test for heteroskedasticity
_, p_value, _, _ = het_white(residuals, model.model.exog)
#print(f'White test p-value: {p_value}')

In [ ]:
#| include: false

# Extracting regression components
coefficients = model.params.values[1:]  # Exclude intercept
errors = model.bse.values[1:]           # Standard errors (excluding intercept)
coef_names = X.columns.tolist()         # Predictor variable names

# Print extracted values for confirmation
print("Coefficients:", coefficients)
print("Standard Errors:", errors)
print("Predictor Names:", coef_names)

In [ ]:
# #| echo: false



# Custom color palette
gradient_colors = ["#B39DDB", "#FFAB91", "#81D4FA"]  # Light Purple, Light Orange, Light Blue

# Updated coefficient names for correct alignment
coef_names = ["% change in Airbnbs", "Vacant properties (%)", "D. to centre (km)"]

# Set up plot style
sns.set(style="white", font_scale=1.1)

# Create a 2x2 grid
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# ----------------- Plot 1: Residuals vs Fitted Values -----------------
axes[0, 0].scatter(fitted_values, residuals, color=gradient_colors[0], alpha=0.7, edgecolor="white", s=50)
axes[0, 0].axhline(0, color="black", linestyle="--", linewidth=1)
axes[0, 0].set_title("Residuals vs Fitted Values", fontsize=13, fontweight="bold")
axes[0, 0].set_xlabel("Fitted Values")
axes[0, 0].set_ylabel("Residuals")

# ----------------- Plot 2: Histogram of Residuals -----------------
sns.histplot(
    residuals,
    kde=True,  # Add KDE curve
    ax=axes[0, 1],
    color=gradient_colors[1],  # Histogram bars
    edgecolor="black",
    bins=20,
    line_kws={"color": "black", "linewidth": 2.5}  # Style KDE line (black and bold)
)
axes[0, 1].set_title("Distribution of Residuals", fontsize=13, fontweight="bold")
axes[0, 1].set_xlabel("Residuals")
axes[0, 1].set_ylabel("Density")

# ----------------- Plot 3: Q-Q Plot -----------------
qq = sm.qqplot(
    residuals,
    line="45",
    ax=axes[1, 0],
    markerfacecolor="#87CEEB",  # Light blue markers
    markeredgecolor="None",
    alpha=0.7
)

# Style the 45-degree reference line
for line in axes[1, 0].lines:
    if line.get_linestyle() == "-":  
        line.set_color("black")      
        line.set_linestyle("--")
        line.set_linewidth(2)
        break

axes[1, 0].set_title("Q-Q Plot of Residuals", fontsize=13, fontweight="bold")

# ----------------- Plot 4: Regression Coefficients -----------------
bars = axes[1, 1].barh(coef_names[::-1], coefficients[::-1], xerr=errors[::-1], 
                       color=gradient_colors, edgecolor="black", capsize=5)  # Reverse for alignment
axes[1, 1].axvline(0, color="black", linestyle="--", linewidth=1)
axes[1, 1].set_title("Regression Coefficients with 95% Confidence Intervals", fontsize=13, fontweight="bold")
axes[1, 1].set_xlabel("Coefficient Value")

# Adjust label positions to prevent overlap
for bar, coef in zip(bars, coefficients[::-1]):
    width = bar.get_width()
    x_position = width + (0.02 if width > 0 else -0.02)
    alignment = 'left' if width > 0 else 'right'
    y_position = bar.get_y() + bar.get_height() / 2
    axes[1, 1].text(x_position, y_position, f"{width:.3f}", 
                    va='center', ha=alignment, fontsize=10, color="black", 
                    bbox=dict(facecolor="white", edgecolor="none", boxstyle="round,pad=0.2"))

# ----------------- Layout and Final Touches -----------------
plt.tight_layout(pad=2.5)
plt.suptitle("OLS Regression Diagnostics", fontsize=16, fontweight="bold", y=1.02)
plt.show()

In [ ]:
#| include: false

#checking the VIF 
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

# Add constant to the independent variables for VIF calculation
X_with_const = add_constant(X)

# Calculate VIF for each independent variable
vif_data = pd.DataFrame()
vif_data["Variable"] = X_with_const.columns
vif_data["VIF"] = [variance_inflation_factor(X_with_const.values, i) for i in range(X_with_const.shape[1])]

# Display VIF values
print(vif_data)

#No signs of multicolinearity 

The model suggests that for every 1% increase in the number of Airbnb listings, the change in rental prices grew by 0.065%. In areas where Airbnb listings would have doubled, featuring 100% increase, rents would have risen by an average of 6.5%. For tenants, this results in significant price increases: in central London, where average rents are around £2,121 per month, a 6.5% rise would add approximately £138 per month, or £1,656 per year. 

Apart from Airbnb impact, the model results also show that neighborhoods closer to the centre see an additional 0.34% increase in rental prices for every kilometer closer. However, the percentage of vacant dwellings showed little connection to rent changes. 

The story doesn’t end with just the numbers. The analysis also revealed that changes in rental prices are not evenly spread across the city. Tests on the residuals (errors) displayed uneven patterns in the data, and further spatial analysis confirmed significant clustering of residuals. 


In [ ]:
#| include: false
#once I grouped the data by boroughs I realised it was not really showing anything

# ____________ checking morans I for any spatial autocorellation ____

#print(type(Regression_Final1))


# needs to be a geopandas data frame 
Rgdf = gpd.GeoDataFrame(Regression_Final1, geometry='geometry') #Rgdf - residuals gdf
#print(type(Rgdf))

# Adding residuals to the GeoDataFrame
Rgdf['residuals'] = residuals


Rgdf = Rgdf.set_crs(epsg=27700)

#print(Rgdf.crs)


from esda import Moran_Local

# ___________ Morans I ____________

# Queen Weights
w = Queen.from_dataframe(Rgdf, use_index=True)
w.transform = 'R'


mi = esda.Moran(Rgdf['residuals'], w)
print(f"{mi.I:0.4f}")
print(f"{mi.p_sim:0.4f}")

#moran_scatterplot(mi)

# ________ local Morans I on the residuals ___________

lisa = esda.Moran_Local(Rgdf.residuals, w)
# Break observations into significant or not
Rgdf['sig'] = lisa.p_sim < 0.05
# Store the quadrant they belong to
Rgdf['quad'] = lisa.q
Rgdf[['LAName','residuals','sig','quad']].sample(5, random_state=42)

#print(Rgdf)

plot_local_autocorrelation(lisa, Rgdf, 'residuals')

In [ ]:
# #| echo: false

# Set normalization for residuals
vmin = Rgdf['residuals'].min()
vmax = Rgdf['residuals'].max()
norm = TwoSlopeNorm(vmin=vmin, vcenter=0, vmax=vmax)

# Plot the residuals with axes and consistent colorbar
fig, ax = plt.subplots(1, 1, figsize=(10, 8))

# Choropleth map
Rgdf.plot(
    column='residuals',
    cmap='coolwarm',
    norm=norm,
    edgecolor="black",
    linewidth=0.5,
    legend=True,
    legend_kwds={'label': "Residual Value", 'orientation': "horizontal"},
    ax=ax
)

# Add axes labels
ax.set_title("Spatial Distribution of Residuals", fontsize=12, fontweight="bold")
ax.set_xlabel("Longitude", fontsize=10)
ax.set_ylabel("Latitude", fontsize=10)

# Force proper axis limits for clean layout
ax.set_xlim(Rgdf.total_bounds[0], Rgdf.total_bounds[2])
ax.set_ylim(Rgdf.total_bounds[1], Rgdf.total_bounds[3])

# Adjust layout
plt.tight_layout()
plt.show()

To build on this analysis, future work could explore how neighborhoods influence each other by using tools that better capture these spatial relationships, including Spatial Lag or Spatial Error. Adding more information, including income levels and housing availability, could also help explain why rent changes happen in some places but not others. Additionally, a more robust regression model can be undertaken that accounts for the uneven spread of residuals.

### Potential analysis expansion:
Below are the potential additional variables that could be tested in the future in multiple linear regression:
* Social housing waiting times;
* No-fault evictions;
* Total new rough sleepers;
* Average long-term room rental prices (e.g. from Spareroom);
* % or N of Houses of Multiple Occupancy;
* % or N of flatshare households;
* N of Property guardians;
* N of rooms in airbnb compared to n of homes on Airbnb.

### Implications to the regulation of STL in London:
**Promote casual STLs**
Following the EDA, enforcing the 90-day rule remains crucial. Multi-listing hosts with large portfolios should be closely monitored for compliance, as they may duplicate listings to bypass platform restrictions. A centralized STL registry, as proposed in February 2024, would enhance transparency, ensure compliance, enable local councils to monitor STL density and trends, and effectively distinguish between casual and professional hosts.

**Encourage private rooms not entire homes**
Given the high concentration of listings, increased pressure on the rental market, and the 0.34% rent increase per kilometer closer to the city center, we propose implementing zoning restrictions in high-demand central boroughs such as Westminster, Camden, and Kensington & Chelsea, and in other areas based on rental market vulnerability, housing supply shortages, and local socioeconomic conditions. We suggest using spatial models (e.g., Spatial Lag/Error) to predict areas at risk of STL saturation and proactively apply regulation. These measures could cap the total number of listings or limit entire home rentals to encourage private room listings or overall reduction of the number of listings.

**Address-based restrictions**
The limitations of anonymized data restrict analysis of property types used for short-term lets. Once the STL registry is operational, regulations could target specific addresses, such as affordable housing, social housing, and newly built properties, to preserve their intended purpose of addressing housing needs rather than catering to tourists.

**Redirect STL-generated taxes or fines**
There is a potential to address declining homeownership through redirecting STL taxes or fines into affordable housing initiatives, supporting vulnerable renters and first-time buyers. Additionally, introducing an STL tax or levy in these areas could generate revenue to support affordable housing and community programs.

**Long-term rent should be made more profitable**
Greater support is needed to encourage property owners to prioritize long-term rentals over short-term lets. Currently, STLs are widely promoted as the most profitable option. Policy solutions could include requiring hosts and platforms to conduct Social Impact Assessments to evaluate STL impacts on local housing markets during planning approvals. Additionally, adjustments to the tax regime for long-term rentals, funded through redirected STL taxes, could provide further incentives.

**The landlord should have a choice**
Under the new rules, homeowners will be able to let their primary residence for up to 90 nights annually without planning permission, and dedicated short-term lets will be automatically reclassified into a new use class. This approach does not provide landlords sufficient time to assess the revenue implications, particularly if proposed incentives for long-term rentals are introduced. The government should reconsider the automatic reclassification of short-term lets.

## References